In [62]:
import torch

# import qpth.solvers.dynamic.solve as dynamic_solver
# from qpth.util import get_sizes

def get_sizes(G, A=None):
    if G.dim() == 2:
        nineq, nz = G.size()
        nBatch = 1
    elif G.dim() == 3:
        nBatch, nineq, nz = G.size()
    if A is not None:
        neq = A.size(1) if A.nelement() > 0 else 0
    else:
        neq = None
    # nBatch = batchedTensor.size(0) if batchedTensor is not None else None
    return nineq, nz, neq, nBatch

In [63]:
def forward(H, c, A, b, verbose=0, maxIter=100, dt=0.2):
    nineq, nz, neq, nBatch = get_sizes(A)
    A_T = torch.transpose(A, -1,1)
    H_I = torch.inverse(H)

    AH_I = A.matmul(H_I)
    D = - AH_I.matmul(A_T)
    d = - (b.unsqueeze(2) + AH_I.matmul(c.unsqueeze(2)))

    y = torch.zeros(nBatch, nineq, 1)
    zeros = torch.zeros(nBatch, nineq, 1)

    for _ in range(maxIter):
        dy = dt * (D.matmul(y) + d)
        dy = torch.max(y + dy, zeros) - y
        y.add_(dy)

    zhat = - H_I.matmul(c.unsqueeze(2) + A_T.matmul(y))
    slacks = b.unsqueeze(2) - A.matmul(zhat)
    
    return zhat, y, slacks


In [64]:
# Example data from: https://www.sciencedirect.com/science/article/pii/S0307904X1000380X#f0010

c = torch.tensor([-30, -30], dtype=torch.float).unsqueeze(0)
b = torch.tensor([35/12, 35/2, 5, 5, 0, 0], dtype=torch.float).unsqueeze(0)
A = torch.tensor([
    [5/12, -1],
    [5/2, 1],
    [-1, 0],
    [0, 1],
    [-1, 0],
    [0, -1]
], dtype=torch.float).unsqueeze(0)
H = torch.tensor([[2,1],[1,2]], dtype=torch.float).unsqueeze(0)

print(c)
print(b)
print(A)
print(H)

tensor([[-30., -30.]])
tensor([[ 2.9167, 17.5000,  5.0000,  5.0000,  0.0000,  0.0000]])
tensor([[[ 0.4167, -1.0000],
         [ 2.5000,  1.0000],
         [-1.0000,  0.0000],
         [ 0.0000,  1.0000],
         [-1.0000,  0.0000],
         [ 0.0000, -1.0000]]])
tensor([[[2., 1.],
         [1., 2.]]])


In [65]:
zhat, y, slacks = forward(H, c, A, b, maxIter=100)

print(zhat)
print(y)
print(slacks)

tensor([[[5.0000],
         [5.0000]]])
tensor([[[0.0000],
         [6.0000],
         [0.0000],
         [9.0000],
         [0.0000],
         [0.0000]]])
tensor([[[ 5.8333e+00],
         [-3.8147e-06],
         [ 1.0000e+01],
         [-4.7684e-06],
         [ 5.0000e+00],
         [ 5.0000e+00]]])
